In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%cd /content/gdrive/MyDrive/Colab Notebooks/FeAl_experiment

/content/gdrive/MyDrive/Colab Notebooks/FeAl_experiment


In [6]:
pip install fast-histogram

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Step_1 genearting experimental zSDMs of each elemental pair

In [4]:
import numpy as np
from scipy import spatial
import pandas as pd 
import os
import datetime
from itertools import product
from fast_histogram import histogram2d as fast_histogram2d 
from joblib import Parallel, delayed
from tqdm import tqdm
from os import listdir
import re

In [14]:
def ZSDM(count):
    # zSDM_simu_0_pre = np.zeros((int(0.69/0.015*2+1), 2))
    # zSDM_simu_1_pre = np.zeros((int(0.69/0.015*2+1), 2))
    # count = 1
    # temp_starttime = datetime.datetime.now()
    data_voxel_sphere = element_chosen[index_voxel_sphere[count,],:]
    if len(data_voxel_sphere) == 0:
        # print ('blank image')
        x_zSDM = np.arange(-0.69, 0.7, (0.69*2+0.015)/93).reshape((93, 1))
        y_zSDM = np.zeros((93, 1))
        zSDM_simu = np.concatenate((x_zSDM, y_zSDM), axis=1)
        return zSDM_simu 
        # fig2D = plt.figure(figsize=(4,4))
        # ax2D = fig2D.add_subplot(111)
        # plt.axis('off')
        # if element == 'Fe':
        #     if count < 10: 
        #         fig2D.savefig('Results_ZSDMs\Fe_Fe\Fe_Fe_00000%s.png'%(count),dpi=SDM_bins)
        #     elif count<100:
        #         fig2D.savefig('Results_ZSDMs\Fe_Fe\Fe_Fe_0000%s.png'%(count),dpi=SDM_bins)                                    
        #     elif count<1000:
        #         fig2D.savefig('Results_ZSDMs\Fe_Fe\Fe_Fe_000%s.png'%(count),dpi=SDM_bins)                                                                        
        #     elif count<10000:
        #         fig2D.savefig('Results_ZSDMs\Fe_Fe\Fe_Fe_00%s.png'%(count),dpi=SDM_bins)  
        #     elif count<100000:
        #         fig2D.savefig('Results_ZSDMs\Fe_Fe\Fe_Fe_0%s.png'%(count),dpi=SDM_bins)  
        # else:
        #     if count < 10: 
        #         fig2D.savefig('Results_ZSDMs\Al_Al\Al_Al_00000%s.png'%(count),dpi=SDM_bins)
        #     elif count<100:
        #         fig2D.savefig('Results_ZSDMs\Al_Al\Al_Al_0000%s.png'%(count),dpi=SDM_bins)                                    
        #     elif count<1000:
        #         fig2D.savefig('Results_ZSDMs\Al_Al\Al_Al_000%s.png'%(count),dpi=SDM_bins)                                                                        
        #     elif count<10000:
        #         fig2D.savefig('Results_ZSDMs\Al_Al\Al_Al_00%s.png'%(count),dpi=SDM_bins)  
        #     elif count<100000:
        #         fig2D.savefig('Results_ZSDMs\Al_Al\Al_Al_0%s.png'%(count),dpi=SDM_bins) 
        # plt.close()   
        # return
    #Looking for neighbor quickly using tree.query_ball_point
    SDM = np.zeros([SDM_bins,SDM_bins])
    x_tot = [];
    y_tot = [];
    num_in_SDM = 0;
    max_cand =0
    cand = tree.query_ball_point(data_voxel_sphere, 1.5,return_sorted=False)
    for list in cand:
        num_in_SDM += len(list);
        if (len(list) > max_cand):
            max_cand = len(list);
    x_tot = np.zeros([num_in_SDM,], dtype = np.float32)
    y_tot = np.zeros([num_in_SDM,], dtype = np.float32)
    x = np.zeros([max_cand,], dtype = np.float32)   
    y = np.zeros([max_cand,], dtype = np.float32)   
    
    start = 0;
    i = 0;
    for list in cand:
        length = len(list)
        x_tot[start:(start+length)] = np.ndarray.__sub__(element_chosen[list,0],data_voxel_sphere[i,0]);
        y_tot[start:(start+length)] = np.ndarray.__sub__(element_chosen[list,2],data_voxel_sphere[i,2]);
        i += 1
        start = start+length;
    notzero = (x_tot!=0)*(y_tot!=0);
    SDM = fast_histogram2d(y_tot[notzero],x_tot[notzero], range = [[-1.5,1.5],[-1.5,1.5]],  bins=SDM_bins)
    #Leigh: https://iscinumpy.gitlab.io/post/histogram-speeds-in-python/ <-- this offers some clues, I used fast histogram mentioned there.
    #%%z-SDMs curve
    y_zSDM = SDM.sum(axis=1).reshape((200, 1))
    # sum_y_zSDM = sum(y_zSDM)
    # y_fre = y_zSDM/sum_y_zSDM*100
    # y_fre = np.array(y_fre).reshape((len(y_fre), 1))
            
    # # scaler = StandardScaler()
    # scaler = MinMaxScaler() 
    # scaler.fit(y_fre)
    # y_fre_scale = scaler.transform(y_fre)          #normalization
    
    x_zSDM = np.arange(-1.5, 1.5, 3/200).reshape((200, 1))
    # zSDM_simu = np.zeros((200, 1))
    zSDM_simu = np.concatenate((x_zSDM, y_zSDM), axis=1)
    zSDM_simu_index = np.where((zSDM_simu[:,0]>=-0.7) & (zSDM_simu[:,0]<0.7))
    zSDM_simu_index_array = np.array(zSDM_simu_index).reshape(-1, 1)
    zSDM_simu_part = zSDM_simu[zSDM_simu_index_array[0, 0]:zSDM_simu_index_array[-1, 0]+1, ]
    
    # #Plot
    # fig2D = plt.figure(figsize=(4,4))                
    # ax2D = fig2D.add_subplot(111)   
    # plt.plot(zSDM_simu_part[: ,0], zSDM_simu_part[: ,1])
    # # ax2D.set_aspect(1)
    # plt.xlim((-0.7,0.7))
    # # plt.ylim((0.0,1.0))
    # # ax2D.set_xticks([])
    # # ax2D.set_yticks([])
    # # plt.axis('off')
    # # plt.subplots_adjust(0,0,1,1) 
    # plt.close() 
    # if element_num == 1:
    #     zSDM_simu_0_pre[:, count*2:(count*2+2)] = zSDM_simu_part   # know where to put zSDM
    #     # if count < 10: 
    #     #     fig2D.savefig('Results_ZSDMs\Fe_Fe\Fe_Fe_00000%s.png'%(count),dpi=SDM_bins)
    #     # elif count<100:
    #     #     fig2D.savefig('Results_ZSDMs\Fe_Fe\Fe_Fe_0000%s.png'%(count),dpi=SDM_bins)                                    
    #     # elif count<1000:
    #     #     fig2D.savefig('Results_ZSDMs\Fe_Fe\Fe_Fe_000%s.png'%(count),dpi=SDM_bins)                                                                        
    #     # elif count<10000:
    #     #     fig2D.savefig('Results_ZSDMs\Fe_Fe\Fe_Fe_00%s.png'%(count),dpi=SDM_bins)  
    #     # elif count<100000:
    #     #     fig2D.savefig('Results_ZSDMs\Fe_Fe\Fe_Fe_0%s.png'%(count),dpi=SDM_bins)  
    # else:
    #     zSDM_simu_1_pre[:, count*2:(count*2+2)] = zSDM_simu_part
    #     # if count < 10: 
    #     #     fig2D.savefig('Results_ZSDMs\Al_Al\Al_Al_00000%s.png'%(count),dpi=SDM_bins)
    #     # elif count<100:
    #     #     fig2D.savefig('Results_ZSDMs\Al_Al\Al_Al_0000%s.png'%(count),dpi=SDM_bins)                                    
    #     # elif count<1000:
    #     #     fig2D.savefig('Results_ZSDMs\Al_Al\Al_Al_000%s.png'%(count),dpi=SDM_bins)                                                                        
    #     # elif count<10000:
    #     #     fig2D.savefig('Results_ZSDMs\Al_Al\Al_Al_00%s.png'%(count),dpi=SDM_bins)  
    #     # elif count<100000:
    #     #     fig2D.savefig('Results_ZSDMs\Al_Al\Al_Al_0%s.png'%(count),dpi=SDM_bins)   
    # # temp_endtime = datetime.datetime.now()
    # # print ('The running time of SDM %s/%s= '%(count, len(index_voxel_sphere)), temp_endtime-temp_starttime)   
    return zSDM_simu_part

def atom_filter(x, Atom_range):
    Atom_total = pd.DataFrame()
    for i in range(len(Atom_range)):
        Atom = x[x['Da'].between(Atom_range['lower'][i], Atom_range['upper'][i], inclusive=True)]
        Atom_total = Atom_total.append(Atom)
        # Count_Atom= len(Atom_total['Da'])   
    return Atom_total[['x','y','z']]

def read_rrng(f):
    rf = open(f,'r').readlines()
    patterns = re.compile(r'Ion([0-9]+)=([A-Za-z0-9]+).*|Range([0-9]+)=(\d+.\d+) +(\d+.\d+) +Vol:(\d+.\d+) +([A-Za-z:0-9 ]+) +Color:([A-Z0-9]{6})')
    ions = []
    rrngs = []
    for line in rf:
        m = patterns.search(line)
        if m:
            if m.groups()[0] is not None:
                ions.append(m.groups()[:2])
            else:
                rrngs.append(m.groups()[2:])
    ions = pd.DataFrame(ions, columns=['number','name'])
    ions.set_index('number',inplace=True)
    rrngs = pd.DataFrame(rrngs, columns=['number','lower','upper','vol','comp','colour'])
    rrngs.set_index('number',inplace=True) 
    rrngs[['lower','upper','vol']] = rrngs[['lower','upper','vol']].astype(float)
    rrngs[['comp','colour']] = rrngs[['comp','colour']].astype(str)
    return ions, rrngs

def readpos(file_name):
    f = open(file_name, 'rb')
    dt_type = np.dtype({'names':['x', 'y', 'z', 'm'], 
                  'formats':['>f4', '>f4', '>f4', '>f4']})
    pos = np.fromfile(f, dt_type, -1)
    f.close()
    return pos


In [15]:
#%%
if __name__ == "__main__":
    #%% record start time
    starttime = datetime.datetime.now()
    #%% Build output file
    # try:
    #     shutil.rmtree('Results_ZSDMs\Fe_Fe')  
    #     shutil.rmtree('Results_ZSDMs\Al_Al')  
    # except:
    #     print("file 1 does not exist")
    # os.mkdir('Results_ZSDMs\Fe_Fe') 
    # os.mkdir('Results_ZSDMs\Al_Al') 
    #%% Input data
    folder = 'data'
    
    # for filename in tqdm(os.listdir(folder)):    
    #     print(filename)
    #     pos = readpos(folder+'/'+filename)
    #     dpos = pd.DataFrame({'x':pos['x'],
    #                                 'y': pos['y'],
    #                                 'z': pos['z'],
    #                                 'Da': pos['m']})
    #     dpos.to_csv(folder+'/'+'{}.csv'.format(filename), index=False)  

    data_name = [file for file in listdir(folder) if file.endswith('.csv')]
    data_name = data_name[0]
    data = pd.read_csv(folder+'/'+data_name)
    data.columns =['x', 'y', 'z', 'Da']

    
    #Finding element 1 Fe and element 2 Al    
    rrange_file = 'recon_v01.RRNG'
    ions, rrngs = read_rrng(rrange_file)
    element_1_range = rrngs[rrngs['comp']=='Fe:1']
    element_2_range = rrngs[rrngs['comp']=='Al:1']
    # element_3_range = rrngs[rrngs['comp']=='Ni:1']
    
    element_1 = atom_filter(data, element_1_range)
    element_2 = atom_filter(data, element_2_range)
    # element_3 = atom_filter(data, element_3_range)
    
    # element_1 = element_1
    # get max, min of each column    
    # df = pd.DataFrame(data,columns=['a','b','c','d'])  #'a', 'b', 'c', 'd' for x,y,z,m. 
    data_min = data.min()
    data_min['c'], data_min['b'], data_min['a'] = -10, -10, -10  #nm   -175 maximum
    data_max = data.max()
    data_max['c'], data_max['b'], data_max['a'] = 0, 10, 10   #nm
   
    # row_number = data.shape[0]
    # detect_eff = 0.52
    #%%Finding element 1 Fe and element 2 Al
    # element_1 = df.loc[((df['d'] >= 27.832) & (df['d'] <= 28.115)) |
    #                    ((df['d']>=26.929) & (df['d'] <=27.038)) |
    #                    ((df['d']>=28.436) & (df['d'] <=28.542)) |
    #                    ((df['d']>=28.919) & (df['d'] <=29.039)) , ['a','b','c']]  
    # element_2 = df.loc[((df['d'] >= 13.444) & (df['d'] <=13.558)) | 
    #                     ((df['d'] >= 8.99) & (df['d'] <=9.02))  , ['a','b','c']]  
    #Note to use & or | to replace and/or
    element_num=0
    
    #%% scanning parameters 
    voxel = 1   #nm
    stride = 0.5   #nm
    #%%Building sphere heart array
    data_Z_list = list(np.arange(int(data_min['c']), int(data_max['c']), stride))
    data_Y_list = list(np.arange(int(data_min['b']), int(data_max['b']), stride))
    data_X_list = list(np.arange(int(data_min['a']), int(data_max['a']), stride))
    data_sphere_points = np.zeros((1,3))
    for data_Z, data_Y, data_X in product(data_Z_list, data_Y_list, data_X_list):
        if data_Z+voxel > data_max['c'] or data_Y+voxel > data_max['b'] or data_X+voxel > data_max['a']:
            continue
        else:
            temp = np.array([data_X+voxel/2, data_Y+voxel/2, data_Z+voxel/2]).reshape((1,3)) 
            data_sphere_points = np.concatenate((data_sphere_points, temp), axis=0)
    data_sphere_points = data_sphere_points[1:]
    
    for element_chosen in (element_1.values, element_2.values) :  #
        element_num = element_num+1
        print ('The chosen element is ', element_num)
    
        #%%create sphere voxels using tree.query_ball_point
        print('Please wait for a moment')
        tree = []
        temp_starttime = datetime.datetime.now()
        tree = spatial.cKDTree(element_chosen)
        temp_endtime = datetime.datetime.now()
        print ('The running time of spatial.cKDTree = ', temp_endtime-temp_starttime)
        #Yue: approximately 1 min on my computer
        #%%
        index_voxel_sphere = tree.query_ball_point(data_sphere_points, voxel/2) 
        #-1 for all processors
        #%% generating SDMs 
        SDM_bins = 200   #define pixel density
        zSDM_output = np.zeros((int(0.69/0.015*2+1), len(index_voxel_sphere)*2))
        myList = range(0,len(index_voxel_sphere))
        zSDM_output = Parallel(n_jobs=1, verbose=2)(delayed(ZSDM)(i) for i in myList)
  
        # zSDM_simu_0_pre, zSDM_simu_1_pre = zip(*zSDM_output)
        if element_num == 1:
            zSDM_simu_1 = zSDM_output   # know where to put zSDM 
        if element_num == 2:
            zSDM_simu_2 = zSDM_output
        # if element_num == 3:
        #     zSDM_simu_3 = zSDM_output
    #%%save
    # zSDM_simu_0 = zSDM_simu_0[:,1:]
    # zSDM_simu_1 = zSDM_simu_1[:,1:]
    np.save ("zSDM_exp_test_Fe_%s_%s_10_10_1_0.5.npy"%(int(data_min['c']), int(data_max['c'])), zSDM_simu_1)
    np.save ("zSDM_exp_test_Al_%s_%s_10_10_1_0.5.npy"%(int(data_min['c']), int(data_max['c'])), zSDM_simu_2)
    # np.save ("zSDM_exp_test_Ni_%s_%s_5_5_2_2.npy"%(int(data_min['c']), int(data_max['c'])), zSDM_simu_3)
    #%% recording endtime
    endtime = datetime.datetime.now()
    print ('Total running time = ', endtime-starttime)

<ipython-input-14-5f2a3287e669>:128: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  Atom = x[x['Da'].between(Atom_range['lower'][i], Atom_range['upper'][i], inclusive=True)]
<ipython-input-14-5f2a3287e669>:129: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Atom_total = Atom_total.append(Atom)
<ipython-input-14-5f2a3287e669>:128: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  Atom = x[x['Da'].between(Atom_range['lower'][i], Atom_range['upper'][i], inclusive=True)]
<ipython-input-14-5f2a3287e669>:129: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Atom_total = Atom_total.append(Atom)
<ipython-input-14-5f2a3287e669>:128: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated 

The chosen element is  1
Please wait for a moment
The running time of spatial.cKDTree =  0:00:00.709264


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 28899 out of 28899 | elapsed:   19.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


The chosen element is  2
Please wait for a moment
The running time of spatial.cKDTree =  0:00:00.111375


[Parallel(n_jobs=1)]: Done 28899 out of 28899 | elapsed:    3.8s finished


Total running time =  0:00:27.123413


Step_2 experimental data preprocessing

In [16]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import shutil
import os
from scipy.signal import savgol_filter
from scipy.signal import argrelextrema
from joblib import Parallel, delayed
# from Data_preprocessing_v4_1_parallel import *

# #parameters
n_input = 93

In [17]:
################################################################# 
#Functions
################################################################
# Data normalization from 0 to 1 for double column dataframe, returns single column array
def normdata(data):
    
    (len1,w1) = np.shape(data)
    ndata = np.zeros([len1,w1//2])
    for i in range(w1//2):
        ndata[:,i]=(data[:,2*i+1]-min(data[:,2*i+1]))/(max(data[:,2*i+1])-min(data[:,2*i+1]))
    return ndata

# Data normalization from 0 to 1 for single column dataframe
def normdatasingle(data):
    
    (len1,w1) = np.shape(data)
    ndata = np.zeros([len1,w1])
    for i in range(w1):
        ndata[:,i]=(data[:,i]-min(data[:,i]))/(max(data[:,i])-min(data[:,i]))
    return ndata

# Extracting experimental data
def exp_data_processing_parallel (i):
    dim = int(len1/n_input)
    # i=1
    # print ('dim=', dim, 'w1=', w1)
    new1_BG_remove_3 = np.zeros([len1, 1])
    for j in range(dim):
        # for i in range(w1):
        if np.isnan(np.min(nzSDM_exp[n_input*j : n_input*(j+1), i])):
            # print ("Nan")
            new1_BG_remove_3[n_input*j : n_input*(j+1), ] = nzSDM_exp[n_input*j : n_input*(j+1), i].reshape(-1,1)
            continue
        # print ('i=',i, 'j=', j)
        #savgol_filter to smooth the data
        new1 = savgol_filter(nzSDM_exp[n_input*j : n_input*(j+1), i], 15, 8)  #21
        # # Plot smoothed curve    
        # fig2D = plt.figure(figsize=(4,4))                
        # ax2D = fig2D.add_subplot(111)   
        # plt.plot(zSDM_exp_FeFe[:, 0], nzSDM_exp[n_input*j : n_input*(j+1), i], label='Original curve')
        # plt.plot(zSDM_exp_FeFe[:, 0], new1[:,], label='Smoothed curve')
        # plt.legend(loc='upper right')
        
        # Background substraction based on Moody 2009
        new1_BG_remove = np.zeros([n_input, 1])
        new1_BG_remove_2 = np.zeros([n_input, 1])
        new1_BG_remove[:,0] = new1[0:n_input, ]
        new1_BG_remove_2[0,0], new1_BG_remove_2[n_input-1,0] = new1[0,], new1[n_input-1,] 
        # new1_BG_remove_2[0,0], new1_BG_remove_2[n_input-1,0] = min(new1[0:n_input, ]), min(new1[0:n_input, ])
        
        # local minmial
        localmin = argrelextrema(new1, np.less)
        localmin_array = np.array(localmin)
        localmin_value = localmin_array[np.where(localmin_array>int(n_input/2))]
        # if new1[0,] == max(new1[0:n_input, ]):
        #     new1_BG_remove_2[0,0] = min(new1[0:n_input, ])
        # if new1[n_input-1,] == max(new1[0:n_input, ]):
        #     new1_BG_remove_2[n_input-1,0] = min(new1[0:n_input, ])  
    
        try:
            localmin_value[0,]
            ite_1=0
            while max(new1_BG_remove[int(0.8*n_input/2):int(2.3*n_input/4), 0]) > new1[localmin_value[0, ], ]:
                for m in range (1, n_input-1):
                    new1_BG_remove_2[m,] = min(new1_BG_remove[m,], (new1_BG_remove[m-1,] + new1_BG_remove[m+1,])/2)
                new1_BG_remove = new1_BG_remove_2
                # # # Plot background curve
                if ite_1 == 1000:
                    # print("Iteration is above 1000")
                    break
                # fig2D = plt.figure(figsize=(4,4))                
                # ax2D = fig2D.add_subplot(111)  
                # # plt.plot(zSDM_exp_FeFe[:, 0], nzSDM_exp[n_input*j : n_input*(j+1), i], label='Original curve')
                # plt.plot(zSDM_exp_FeFe[:, 0], new1[:,], label='Smoothed curve')
                # plt.plot(zSDM_exp_FeFe[:, 0], new1_BG_remove[:,], label='Background curve')
                ite_1=ite_1+1
                
            # print('Iteration number is ', ite_1)
            # # Plot background curve
            # fig2D = plt.figure(figsize=(4,4))                
            # ax2D = fig2D.add_subplot(111)  
            # plt.plot(zSDM_exp_FeFe[:, 0], nzSDM_exp[n_input*j : n_input*(j+1), i], label='Original curve')
            # plt.plot(zSDM_exp_FeFe[:, 0], new1[:,], label='Smoothed curve')
            # plt.plot(zSDM_exp_FeFe[:, 0], new1_BG_remove[:,], label='Background curve')
            # plt.legend(loc='upper right')
        except:
            new1_BG_remove = np.zeros((n_input, 1))
        # Data after background substraction
        new1_BG_remove_3[n_input*j : n_input*(j+1), ] = normdatasingle(new1.reshape(n_input, 1)-new1_BG_remove)  
        # Plot data after background substraction
        # if save_newexpZSDMs == True:
        #     fig2D = plt.figure(figsize=(4,4))                
        #     ax2D = fig2D.add_subplot(111) 
        #     plt.plot(zSDM_exp_FeFe[:, 0], nzSDM_exp[n_input*j : n_input*(j+1), i], label='Original curve') 
        #     plt.plot(zSDM_exp_FeFe[:, 0], new1_BG_remove[:,], label='Background curve')
        #     plt.plot(zSDM_exp_FeFe[:, 0], new1_BG_remove_3[:, j*w1+i], label='Curve after BR') 
        #     plt.legend(loc='upper right')
        #     plt.close()  
        #     if j==0:
        #         fig2D.savefig('Results_newexpZSDMs/Newexp_ZSDM_'+image_name_1+'_%s.png'%(i), dpi=92)
        #     if j==1:
        #         fig2D.savefig('Results_newexpZSDMs/Newexp_ZSDM_'+image_name_2+'_%s.png'%(i), dpi=92)
        #     # if j==2:
        #     #     fig2D.savefig('Results_newexpZSDMs/Newexp_ZSDM_'+image_name_3+'_%s.png'%(i), dpi=92)    
 
    return new1_BG_remove_3


In [20]:
################################################################
# Test using experimental data
################################################################
# test_dir = 'test/'   #in one folder

################################################################
# Load data and preprocess
################################################################

element_1_name = "FeAl_FeFe"
element_2_name = "FeAl_AlAl"


zSDM_exp_element_1 = np.load('zSDM_exp_test_Fe_%s_%s_10_10_1_0.5.npy'%(int(data_min['c']), int(data_max['c'])))
zSDM_exp_element_2 = np.load('zSDM_exp_test_Al_%s_%s_10_10_1_0.5.npy'%(int(data_min['c']), int(data_max['c'])))


dim = zSDM_exp_element_1.shape[0]*zSDM_exp_element_2.shape[2]
zSDM_exp_element_1_2d = np.reshape(np.transpose(zSDM_exp_element_1,(1,0,2)),(-1, dim))
zSDM_exp_element_2_2d = np.reshape(np.transpose(zSDM_exp_element_2,(1,0,2)),(-1, dim))
# zSDM_exp_Ni_2d = np.reshape(np.transpose(zSDM_exp_Ni,(1,0,2)),(-1, dim))
# Data normalization
nzSDM_exp_element_1 = normdata(zSDM_exp_element_1_2d)
nzSDM_exp_element_2 = normdata(zSDM_exp_element_2_2d)
# nzSDM_exp_NiNi = normdata(zSDM_exp_Ni_2d)
nzSDM_exp= np.concatenate((nzSDM_exp_element_1, nzSDM_exp_element_2), axis=0) 
save_newexpZSDMs = False
# Build ouptfile
if save_newexpZSDMs == True:
    try:
        shutil.rmtree('Results_newexpZSDMs')
    except:
        print("file does not exist")
    os.mkdir('Results_newexpZSDMs')

# Data background substraction
(len1,w1) = np.shape(nzSDM_exp)
dim = int(len1/n_input)

myList = range(0,w1)
nzSDM_exp_preprocessing_parallel = Parallel(n_jobs=-1, verbose=2)(delayed(exp_data_processing_parallel)(i) for i in myList)
nzSDM_exp_preprocessing_parallel_2 = np.array(nzSDM_exp_preprocessing_parallel)
np.save('nzSDM_exp_preprocessing_2_%s_%s_10_10_1_0.5.npy'%(int(data_min['c']), int(data_max['c'])), nzSDM_exp_preprocessing_parallel_2)


<ipython-input-17-4ba50d956e6b>:10: RuntimeWarning: invalid value encountered in true_divide
  ndata[:,i]=(data[:,2*i+1]-min(data[:,2*i+1]))/(max(data[:,2*i+1])-min(data[:,2*i+1]))
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 3689 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 6706 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 10148 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 16074 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done 22392 tasks      | elapsed: 10.3min
[Parallel(n_jobs=-1)]: Done 28899 out of 28899 | elapsed: 12.9min finished


Step_3 Predictions

In [21]:
#######################
#Test results
nzSDM_exp_preprocessing_parallel_2 = np.load('nzSDM_exp_preprocessing_2_%s_%s_10_10_1_0.5.npy'%(int(data_min['c']), int(data_max['c'])))
x_exp = np.reshape(nzSDM_exp_preprocessing_parallel_2, (w1, len1))
x_exp_dim = np.stack((x_exp[:, :n_input], x_exp[:, n_input:]), axis=2)

y_exp_prediction_0, y_exp_prediction_1, y_exp_prediction_2 = np.zeros((w1, 1)), np.zeros((w1, 1)), np.zeros((w1, 1))

for k in range(5):
    #calling model 
    print('k=', k)
    model = tf.keras.models.load_model('saved_models_1CNN_layer_kernel_size_10_neuro_256_new_data_3class_multi_models/model_%s.h5'%(k)) 
    model.summary()
     
    # # Test data
    y_exp_prediction_single = model.predict(x_exp_dim[:,:,:])
    y_exp_prediction_0 = np.concatenate((y_exp_prediction_0, y_exp_prediction_single[:, 0].reshape(-1,1)), axis=1)
    y_exp_prediction_1 = np.concatenate((y_exp_prediction_1, y_exp_prediction_single[:, 1].reshape(-1,1)), axis=1)
    y_exp_prediction_2 = np.concatenate((y_exp_prediction_2, y_exp_prediction_single[:, 2].reshape(-1,1)), axis=1)
y_exp_prediction_0, y_exp_prediction_1, y_exp_prediction_2 = y_exp_prediction_0[:, 1:], y_exp_prediction_1[:, 1:], y_exp_prediction_2[:, 1:]
y_exp_prediction_0_ave, y_exp_prediction_0_std = np.mean(y_exp_prediction_0, axis=1), np.std(y_exp_prediction_0, axis=1)
y_exp_prediction_1_ave, y_exp_prediction_1_std = np.mean(y_exp_prediction_1, axis=1), np.std(y_exp_prediction_1, axis=1)
y_exp_prediction_2_ave, y_exp_prediction_2_std = np.mean(y_exp_prediction_2, axis=1), np.std(y_exp_prediction_2, axis=1)
y_exp_predictions = np.concatenate((y_exp_prediction_0_ave.reshape(-1,1), y_exp_prediction_1_ave.reshape(-1,1), y_exp_prediction_2_ave.reshape(-1,1)), axis=1)

np.save('FeAl_%s_%s_10_10_1_0.5_test_data.npy'%(int(data_min['c']), int(data_max['c'])), y_exp_predictions)
    

k= 0
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 84, 64)            1344      
                                                                 
 dropout (Dropout)           (None, 84, 64)            0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 42, 64)           0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 2688)              0         
                                                                 
 dense (Dense)               (None, 256)               688384    
                                                                 
 dense_1 (Dense)             (None, 3)                 771       
                                                   

Step 4 B2 CSRO extraction

In [22]:
import numpy as np
# import matplotlib.pyplot as plt
from scipy import spatial
import pandas as pd
# import random as rd
# import math
# from mpl_toolkits.mplot3d import Axes3D
# from matplotlib.colors import BoundaryNorm
# from matplotlib.ticker import MaxNLocator
# import matplotlib as mpl
import datetime
from os import listdir
from itertools import product
from joblib import Parallel, delayed

In [26]:
#%% Function
def SRO_extract(i_threshold):
    lower, upper = threshold+i_threshold*ite, threshold+(i_threshold+1)*ite
    print('Threshold is from ', lower, ' to ', upper)
    dspp_filter = dspp.loc[(dspp['d'] > lower) & (dspp['d'] <=upper) , ['a','b','c','d']]  
    tree = []
    tree = spatial.cKDTree(df_1[['a','b','c']])
    index_voxel_sphere = tree.query_ball_point(dspp_filter[['a','b','c']], stride/2*1.414)     
    myList = range(0,len(index_voxel_sphere))
    pre_extract_v1 = pd.DataFrame(np.arange(4).reshape(1,4),columns=['a','b','c','d'])
    for i in myList:
        df_2 = df_1.iloc[index_voxel_sphere[i,]]
        pre_extract_v1 = pre_extract_v1.append(df_2)
    pre_extract_v1 = pre_extract_v1[1:].drop_duplicates().values
    np.savetxt('pre_extract\\'+ sro_type +
               '_10_10_%s_%snm_above_%s_%s_win_%s_stride_%s.csv'
               %(int(data_min['c']), int(data_max['c']), lower, upper, voxel, stride) 
               , pre_extract_v1, delimiter=',', fmt='%.3f')
    return pre_extract_v1


def SRO_extract_only_lower(lower):
    print('Threshold is above ', lower)
    dspp_filter = dspp.loc[(dspp['d'] > lower) , ['a','b','c','d']]  
    tree = []
    tree = spatial.cKDTree(df_1[['a','b','c']])
    index_voxel_sphere = tree.query_ball_point(dspp_filter[['a','b','c']], stride/2*1.414)     
    myList = range(0,len(index_voxel_sphere))
    pre_extract_v1 = pd.DataFrame(np.arange(4).reshape(1,4),columns=['a','b','c','d'])
    for i in myList:
        df_2 = df_1.iloc[index_voxel_sphere[i,]]
        pre_extract_v1 = pre_extract_v1.append(df_2)
    pre_extract_v1 = pre_extract_v1[1:].drop_duplicates().values
    np.savetxt('pre_extract\\'+ sro_type +
               '_10_10_%s_%snm_above_%s_win_%s_stride_%s.csv'
               %(int(data_min['c']), int(data_max['c']), lower, voxel, stride) 
               , pre_extract_v1, delimiter=',', fmt='%.3f')
    return pre_extract_v1

In [27]:
#%% Input APT data
folder = 'data'

# for filename in tqdm(os.listdir(folder)):    
#     print(filename)
#     pos = readpos(folder+'/'+filename)
#     dpos = pd.DataFrame({'x':pos['x'],
#                                 'y': pos['y'],
#                                 'z': pos['z'],
#                                 'Da': pos['m']})
#     dpos.to_csv(folder+'/'+'{}.csv'.format(filename), index=False)  

data_name = [file for file in listdir(folder) if file.endswith('.csv')]
data_name = data_name[0]
df_1 = pd.read_csv(folder+'/'+data_name)
df_1.columns =['a','b','c','d']
# #%% get max, min of each column
# data_min = df_1.min()
# data_min['c'], data_min['b'], data_min['a'] = -175, -10, -10  #nm
# data_max = df_1.max()
# data_max['c'], data_max['b'], data_max['a'] = 0, 10, 10   #nm

#Inout the probability of each voxel
data_predictions = np.load('FeAl_%s_%s_10_10_1_0.5_test_data.npy'%(int(data_min['c']), int(data_max['c'])))
# row_number = df_1.shape[0]
# detect_eff = 0.52
n_dim = data_predictions.shape[1] #0 BCC 1 B2 2 D03
dict = {'B2': 1} #dict = {'B2': 1, 'D03': 2}

#%% define voxel and stride 
voxel = 1   #nm   delta
stride = 0.5   #nm   delta_voxels
count = 0*8

#%% save range
threshold = 4.0
ite = 0.5
#%% reshape data_predictions
img_dim_a = int((data_max['a']-data_min['a'])/1)
img_dim_b = int((data_max['b']-data_min['b'])/1)
img_dim_c = int((data_max['c']-data_min['c'])/1)   
conv_shape_a = int((img_dim_a-voxel)/stride)+1
conv_shape_b = int((img_dim_b-voxel)/stride)+1
conv_shape_c = int((img_dim_c-voxel)/stride)+1

conv_probability = np.zeros([conv_shape_c, conv_shape_b, conv_shape_a])   

for sro_type in dict:   
    # print(dict[sro_type])
    conv_probability[:,:,:] = np.reshape(data_predictions[:,dict[sro_type]], (conv_shape_c, conv_shape_b, conv_shape_a))
# print(conv_probability[:,:,:,len1])
#%% calculate the probability of each voxel (stride*stride*4) 
starttime = datetime.datetime.now()

pixel_probability = np.zeros([int(img_dim_c/stride), int(img_dim_b/stride), int(img_dim_a/stride)])    #1
pixel_z = 0
pixel_y = 0
pixel_x = 0
count = 0
intervial= int(voxel/stride)
# print ('count', count)
# print('pixel_x=', pixel_x)
# print('pixel_y=', pixel_y)
# print('pixel_z=', pixel_z)
# for pixel_dim in range (n_dim):
#     print ('Dimension=', pixel_dim)
for data_Z in np.arange(int(data_min['c']), int(data_max['c']), stride):
    if data_Z+stride > data_max['c']:
        continue
    else:
        for data_Y in np.arange(int(data_min['b']),int(data_max['b']), stride):
            if data_Y+stride > data_max['b']:
                continue
            else:
                for data_X in np.arange(int(data_min['a']), int(data_max['a']), stride):
                    if data_X+stride > data_max['a']:
                        continue
                    else:

                        for k in range (pixel_z, pixel_z-intervial, -1):
                            if k < 0 or k >= conv_shape_c: 
                                continue
                            else:
                                for j in range (pixel_y, pixel_y-intervial, -1):
                                    if j < 0 or j >= conv_shape_b:
                                        continue
                                    else:                                       
                                        for i in range (pixel_x, pixel_x-intervial, -1):
                                            if i < 0 or i >= conv_shape_a:
                                                continue
                                            else:
                                            
                                                temp = conv_probability[k, j, i]
                                                pixel_probability[pixel_z, pixel_y, pixel_x] = pixel_probability[pixel_z, pixel_y, pixel_x] + temp
                                                # print ('i=',i, "j=", j, "k=", k)
                                                count = count + 1
                        pixel_probability[pixel_z, pixel_y, pixel_x] = pixel_probability[pixel_z, pixel_y, pixel_x]
                        # print ('count', count)
                        count=0        
                        # print ("------>>")
                        if pixel_x == int(img_dim_a/stride-1):
                            pixel_x = 0  
                        else:
                            pixel_x = pixel_x + 1
                        # print('pixel_x=', pixel_x)
                        # print('pixel_y=', pixel_y)
                        # print('pixel_z=', pixel_z)
                if pixel_y == int(img_dim_b/stride-1):
                    pixel_y = 0                        
                else:
                    pixel_y = pixel_y + 1
                # print('pixel_x=', pixel_x)
                # print('pixel_y=', pixel_y)
                # print('pixel_z=', pixel_z)
        if pixel_z == int(img_dim_c/stride-1):
            pixel_z = 0                        
        else:
            pixel_z = pixel_z + 1
        # print('pixel_x=', pixel_x)
        # print('pixel_y=', pixel_y)
        # print('pixel_z=', pixel_z)
endtime = datetime.datetime.now()
print ('1st running time is', endtime-starttime)
#%% Corresponding the probability of each small voxel with each data point, and then saved into .txt file
starttime = datetime.datetime.now()

dim_a = int((data_max['a']-data_min['a'])/stride)
dim_b = int((data_max['b']-data_min['b'])/stride)
dim_c = int((data_max['c']-data_min['c'])/stride)
pixel_probability_flatten=np.reshape(pixel_probability, (dim_a*dim_b*dim_c, 1))

data_Z_list = list(np.arange(int(data_min['c']), int(data_max['c']), stride))
data_Y_list = list(np.arange(int(data_min['b']), int(data_max['b']), stride))
data_X_list = list(np.arange(int(data_min['a']), int(data_max['a']), stride))

data_sphere_points = np.zeros((1,3))
for data_Z, data_Y, data_X in product(data_Z_list, data_Y_list, data_X_list):
    if data_Z+stride > data_max['c'] or data_Y+stride > data_max['b'] or data_X+stride > data_max['a']:
        continue
    else:
        temp = np.array([data_X+stride/2, data_Y+stride/2, data_Z+stride/2]).reshape((1,3)) 
        data_sphere_points = np.concatenate((data_sphere_points, temp), axis=0)
data_sphere_points = data_sphere_points[1:]
data_sphere_points_probability = np.concatenate((data_sphere_points, pixel_probability_flatten), axis=1)
dspp = pd.DataFrame(data_sphere_points_probability, columns=['a','b','c','d'])

#output using different thresholds
# myList = range(0,3)
# pre_extract = Parallel(n_jobs=-1, verbose=1)(delayed(SRO_extract)(i) for i in myList)

SRO_extract_only_lower(4)  

endtime = datetime.datetime.now()
print ('2nd running time is', endtime-starttime)

1st running time is 0:00:00.410540
Threshold is above  4
2nd running time is 0:00:02.051201


D03 CSRO extraaction

In [28]:
dict = {'D03': 2} #dict = {'B2': 1, 'D03': 2}

#%% save range
threshold = 3.75
ite = 0.5
#%% reshape data_predictions
img_dim_a = int((data_max['a']-data_min['a'])/1)
img_dim_b = int((data_max['b']-data_min['b'])/1)
img_dim_c = int((data_max['c']-data_min['c'])/1)   
conv_shape_a = int((img_dim_a-voxel)/stride)+1
conv_shape_b = int((img_dim_b-voxel)/stride)+1
conv_shape_c = int((img_dim_c-voxel)/stride)+1

conv_probability = np.zeros([conv_shape_c, conv_shape_b, conv_shape_a])   

for sro_type in dict:   
    # print(dict[sro_type])
    conv_probability[:,:,:] = np.reshape(data_predictions[:,dict[sro_type]], (conv_shape_c, conv_shape_b, conv_shape_a))
# print(conv_probability[:,:,:,len1])
#%% calculate the probability of each voxel (stride*stride*4) 
starttime = datetime.datetime.now()

pixel_probability = np.zeros([int(img_dim_c/stride), int(img_dim_b/stride), int(img_dim_a/stride)])    #1
pixel_z = 0
pixel_y = 0
pixel_x = 0
count = 0
intervial= int(voxel/stride)
# print ('count', count)
# print('pixel_x=', pixel_x)
# print('pixel_y=', pixel_y)
# print('pixel_z=', pixel_z)
# for pixel_dim in range (n_dim):
#     print ('Dimension=', pixel_dim)
for data_Z in np.arange(int(data_min['c']), int(data_max['c']), stride):
    if data_Z+stride > data_max['c']:
        continue
    else:
        for data_Y in np.arange(int(data_min['b']),int(data_max['b']), stride):
            if data_Y+stride > data_max['b']:
                continue
            else:
                for data_X in np.arange(int(data_min['a']), int(data_max['a']), stride):
                    if data_X+stride > data_max['a']:
                        continue
                    else:

                        for k in range (pixel_z, pixel_z-intervial, -1):
                            if k < 0 or k >= conv_shape_c: 
                                continue
                            else:
                                for j in range (pixel_y, pixel_y-intervial, -1):
                                    if j < 0 or j >= conv_shape_b:
                                        continue
                                    else:                                       
                                        for i in range (pixel_x, pixel_x-intervial, -1):
                                            if i < 0 or i >= conv_shape_a:
                                                continue
                                            else:
                                            
                                                temp = conv_probability[k, j, i]
                                                pixel_probability[pixel_z, pixel_y, pixel_x] = pixel_probability[pixel_z, pixel_y, pixel_x] + temp
                                                # print ('i=',i, "j=", j, "k=", k)
                                                count = count + 1
                        pixel_probability[pixel_z, pixel_y, pixel_x] = pixel_probability[pixel_z, pixel_y, pixel_x]
                        # print ('count', count)
                        count=0        
                        # print ("------>>")
                        if pixel_x == int(img_dim_a/stride-1):
                            pixel_x = 0  
                        else:
                            pixel_x = pixel_x + 1
                        # print('pixel_x=', pixel_x)
                        # print('pixel_y=', pixel_y)
                        # print('pixel_z=', pixel_z)
                if pixel_y == int(img_dim_b/stride-1):
                    pixel_y = 0                        
                else:
                    pixel_y = pixel_y + 1
                # print('pixel_x=', pixel_x)
                # print('pixel_y=', pixel_y)
                # print('pixel_z=', pixel_z)
        if pixel_z == int(img_dim_c/stride-1):
            pixel_z = 0                        
        else:
            pixel_z = pixel_z + 1
        # print('pixel_x=', pixel_x)
        # print('pixel_y=', pixel_y)
        # print('pixel_z=', pixel_z)
endtime = datetime.datetime.now()
print ('1st running time is', endtime-starttime)
#%% Corresponding the probability of each small voxel with each data point, and then saved into .txt file
starttime = datetime.datetime.now()

dim_a = int((data_max['a']-data_min['a'])/stride)
dim_b = int((data_max['b']-data_min['b'])/stride)
dim_c = int((data_max['c']-data_min['c'])/stride)
pixel_probability_flatten=np.reshape(pixel_probability, (dim_a*dim_b*dim_c, 1))

data_Z_list = list(np.arange(int(data_min['c']), int(data_max['c']), stride))
data_Y_list = list(np.arange(int(data_min['b']), int(data_max['b']), stride))
data_X_list = list(np.arange(int(data_min['a']), int(data_max['a']), stride))

data_sphere_points = np.zeros((1,3))
for data_Z, data_Y, data_X in product(data_Z_list, data_Y_list, data_X_list):
    if data_Z+stride > data_max['c'] or data_Y+stride > data_max['b'] or data_X+stride > data_max['a']:
        continue
    else:
        temp = np.array([data_X+stride/2, data_Y+stride/2, data_Z+stride/2]).reshape((1,3)) 
        data_sphere_points = np.concatenate((data_sphere_points, temp), axis=0)
data_sphere_points = data_sphere_points[1:]
data_sphere_points_probability = np.concatenate((data_sphere_points, pixel_probability_flatten), axis=1)
dspp = pd.DataFrame(data_sphere_points_probability, columns=['a','b','c','d'])

#output using different thresholds
# myList = range(0,3)
# pre_extract = Parallel(n_jobs=-1, verbose=1)(delayed(SRO_extract)(i) for i in myList)

SRO_extract_only_lower(3.75)  

endtime = datetime.datetime.now()
print ('2nd running time is', endtime-starttime)

1st running time is 0:00:01.106275
Threshold is above  3.75


<ipython-input-26-506aa3c8cb94>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pre_extract_v1 = pre_extract_v1.append(df_2)
<ipython-input-26-506aa3c8cb94>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pre_extract_v1 = pre_extract_v1.append(df_2)
<ipython-input-26-506aa3c8cb94>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pre_extract_v1 = pre_extract_v1.append(df_2)
<ipython-input-26-506aa3c8cb94>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pre_extract_v1 = pre_extract_v1.append(df_2)
<ipython-input-26-506aa3c8cb94>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

2nd running time is 0:00:04.332827


<ipython-input-26-506aa3c8cb94>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pre_extract_v1 = pre_extract_v1.append(df_2)
<ipython-input-26-506aa3c8cb94>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pre_extract_v1 = pre_extract_v1.append(df_2)
<ipython-input-26-506aa3c8cb94>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pre_extract_v1 = pre_extract_v1.append(df_2)
<ipython-input-26-506aa3c8cb94>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pre_extract_v1 = pre_extract_v1.append(df_2)
<ipython-input-26-506aa3c8cb94>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver